In [19]:
# Analysis Libraries
import math
import numpy as np
import pandas as pd
import uuid  

# SQL Libs
import pyodbc

#SQL Connection
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=.;'
                      'Database=GraduationProject;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

# # Visualization Libraries
import chart_studio as cs
import plotly.express as ex
import chart_studio.plotly as py
import chart_studio.tools as tls
import plotly.graph_objects as go

# Set-Up My "chartstudio" API Key
cs.tools.set_credentials_file(username='Muhammed_Zidan', api_key='KHPd6eu4kntt7on2vypf')

# APIs Libs
import asyncio
import uvicorn
import requests
from fastapi import FastAPI

# Subject Calcolation Functions

In [20]:
def SubjectIndexing(SubmitionsDf):
    SubQuery = SubmitionsDf[
        ['InstructorEfficiency',
       'CourseUnderstand','InstructorRespect', 'InstructorMaterial',
       'ExamContent', 'AssistantTeacher', 'InstructorRecommendation',
       'CourseRecommendation', 'CourseMarket']]

    # Makeing The Piviot Table
    SubQuery = (pd.pivot_table(pd.melt(SubQuery),index="value",columns="variable", aggfunc='size', fill_value=0)[:6])
    SubQuery = pd.DataFrame(SubQuery).reset_index()
    return SubQuery

### Subject Plot Function

In [21]:
def SubjectMainPagePlot(df, subjId):
    
    # Plot Code
    df = df.query('SubjectId == @subjId').sort_values(by=['Name','AnalysisDate'])
    Dep2 = ex.line(df, x = 'AnalysisDate', y = 'AnalysisRate',\
                   animation_frame = 'Name', markers=True, text='Name',\
                  color='Name')
    Dep2.update_layout(
        yaxis_range =[0, 6],
        title=f'Department performance over time',
        title_x=0.5,
        xaxis_title="Survey Date",
        yaxis_title="Over All Rate",
        legend_title="Subject ID",
        font=dict(
            family="Open Sans", 
            size=14,
            color="RebeccaPurple"
        ),
        showlegend=False
    )

    # Disable Controllers
    Dep2.layout.xaxis.fixedrange = True
    Dep2.layout.yaxis.fixedrange = True
    
    Dep2.show
    # Generate Subject Plot IFRAME
    SubIframe = tls.get_embed(py.plot(Dep2, auto_open=False, filename = subjId))
    
    cursor.execute("""
    UPDATE Subjects
    SET IFrame = ?
    WHERE SubjectId = ?
    """ ,(SubIframe, subjId))
    
    conn.commit()
#     Print The Iframe
    return(SubIframe)

In [22]:
def SubjectPlot(SubmitionsDf, subname):
        
    SubjectName = subname + " " +SubmitionsDf.Title.iloc[0]
    
    questid = list(SubmitionsDf.QuestionnaireId.iloc[0])[0]
    
    # Get Query Size
    SubQuerySize = SubmitionsDf.shape[0]

    # Extract Subject's Questions Columns
    SubQuery = SubmitionsDf[
        
        ['InstructorEfficiency',
       'CourseUnderstand','InstructorRespect', 'InstructorMaterial',
       'ExamContent', 'AssistantTeacher', 'InstructorRecommendation',
       'CourseRecommendation', 'CourseMarket']]

    # Makeing The Piviot Table
    SubQuery = (pd.pivot_table(pd.melt(SubQuery),index="value",columns="variable", aggfunc='size', fill_value=0)[:6]).T

    # Reset Index
    SubQuery = pd.DataFrame(SubQuery).reset_index()

    # Visualization Set-UP
    values = SubQuery.iloc[:, 1:6].values.tolist()

    # Set title text 
    title = SubjectName

    # Set width and height
    width = 900
    height = 800

    # Set colors for the Plot
    background_color = "WhiteSmoke" # Background
    colors = [
        "#de425b",  # "Strongly Agree" bars
        "#ea936d",  # "Agree" bars
        "#fbdbb1",  # "Neutral" bars
        "#b2b264",  # Disagree bar
        "#488f31",  # Strongly Disagree bars
    ]

    # Customize font settings of plot annontations
    questions_font = likert_scale_font = dict(
        family="Open Sans",
        size=14, 
        color="black"
    )
    title_font = dict(family="Open Sans", size=20, color="black")
    percent_font = dict(family="Open Sans", size=16, color="black")

    #### Code to create stacked bar chart begins

    # Define Likert scale labels with formatting
    labels = [
        "<b>Strongly<br>disagree<b>",
        "<b>Disagree<b>",
        "<b>Neutral<b>",
        "<b>Agree<b>",
        "<b>Strongly<br>agree<b>",
    ]

    # Add line breaks to questions after fifth word
    questions = []
    qs = SubQuery.iloc[:,0].tolist()

    questions = [
    'Assistant teacher<br>facilitat learning<br>and clearify course<br>practical part?' ,
    'You recommend<br>this course<br>to other students?' ,
    'Instructor present<br>material clearly that<br>facilitated understanding?' ,
    'This course has<br>high educational<br>impact on the market?' ,
    'Exams and assignments<br>reflective course content?' ,
    'instructor organized,<br>well prepared,<br>and use time efficiently' ,
    'Instructional materials <br>increase your knowledge<br>in the subject matter?' ,
    'Instructor treat<br>students with respect?' ,
    'You recommend<br>the instructor<br>to other students?'
    ]

    # Create a Plotly Graph object
    SubPlot = go.Figure()

    # Create a bar for each question and label with the correct color
    for i in range(0, len(values[0])):
        for xd, yd in zip(values, questions):
            SubPlot.add_trace(
                go.Bar(
                    x=[xd[i]],
                    y=[yd],
                    orientation="h",
                    marker=dict(color=colors[i]),
                )
            )

    # Create a horizontal stacked bar chart
    SubPlot.update_layout(
        title=title,
        title_font=title_font,
        width=width, 
        height=height,
        barmode="stack",
        paper_bgcolor=background_color,
        plot_bgcolor=background_color,
        margin=dict(l=120, r=10, t=140, b=10),
        showlegend=False,
        hovermode=False,
        xaxis=dict(
            showgrid=False,
            showline=False,
            showticklabels=False,
            zeroline=False,
            domain=[0.15, 1],
        ),
        yaxis=dict(
            showgrid=False,
            showline=False,
            showticklabels=False,
            zeroline=False,
        ),
    )

    # Create and add annotations to plot
    annotations = []
    for yd, xd in zip(questions, values):
        # Label the y-axis with the questions
        annotations.append(
            dict(
                xref="paper",
                yref="y",
                x=0.14,
                y=yd,
                xanchor="right",
                text=str(yd),
                font=questions_font,
                showarrow=False,
                align="left",
            )
        )
        # Label the first percentage of the questions on the x-axis
        annotations.append(
            dict(
                xref="x",
                yref="y",
                x=xd[0] / 2,
                y=yd,
                text=str(round(xd[0]/SubQuerySize *100)) + "%",
                font=percent_font,
                showarrow=False,
            )
        )
        # Label the first Likert scale on the top
        if yd == questions[-1]:
            annotations.append(
                dict(
                    xref="x",
                    yref="paper",
                    x=xd[0] / 2,
                    y=1.08,
                    text=labels[0],
                    font=likert_scale_font,
                    showarrow=False,
                )
            )
        space = xd[0]
        for i in range(1, len(xd)):
            # Label the rest of the percentages of the questions on the x-axis
            annotations.append(
                dict(
                    xref="x",
                    yref="y",
                    x=space + (xd[i] / 2),
                    y=yd,
                    text=str(round(xd[i]/SubQuerySize*100)) + "%",
                    font=percent_font,
                    showarrow=False,
                )
            )
            # Label the rest of the Likert scale on the top
            if yd == questions[-1]:
                annotations.append(
                    dict(
                        xref="x",
                        yref="paper",
                        x=space + (xd[i] / 2),
                        y=1.08,
                        text=labels[i],
                        font=likert_scale_font,
                        showarrow=False,
                    )
                )
            space += xd[i]
    SubPlot.update_layout(annotations=annotations)
    
    # Disable Controllers 
    SubPlot.layout.xaxis.fixedrange = True
    SubPlot.layout.yaxis.fixedrange = True
#     SubPlot.show()
    # Generate Subject Plot IFRAME
    SubIframe = tls.get_embed(py.plot(SubPlot,filename= questid, auto_open=False))

    cursor.execute("""
    UPDATE Questionnaires
    SET IFrame = ?
    WHERE QuestionnaireId = ?
    """ ,(SubIframe, questid))
   
    conn.commit()
    
    # Return The Iframe
    return SubIframe

### Subject Conclsion Function

In [23]:
def SubjectConclsion(RatesList):
    GoodConc = []
    BadConc =[]
    # Question 1 
    if RatesList[0] < 3 :
        BadConc.append('The assistant teacher MUST facilitat more the learning and clearify course practical part')
    elif RatesList[0] >= 3 and RatesList[0] < 5:
        GoodConc.append('GOOD Assistant Teacher!')
    else:
        GoodConc.append('PERFECT Assistant Teacher!')

    # Question 2
    if RatesList[1] < 3 :
        BadConc.append("Work on making the course more useful to be recommended")
    elif RatesList[1] >= 3 and RatesList[1] < 5:
        GoodConc.append('The Course Is Recommended KINDA Often!')
    else:
        GoodConc.append('The Course Is Recommended SO MUCH !')
    # Question 3
    if RatesList[2] < 3 :
        BadConc.append("Please present course material in a clear manner because students don't understand well")
    elif RatesList[2] >= 3 and RatesList[2] < 5:
        GoodConc.append('The Course Is Understandable')
    else:
        GoodConc.append('The Course Is Understandable PERFECTLY !')
        
    # Question 4
    if RatesList[3] < 3 :
        BadConc.append("You need to update your course that it can fet the market requirements")
    elif RatesList[3] >= 3 and RatesList[3] < 5:
        GoodConc.append('The Course Is UP-To-Market !')
    else:
        GoodConc.append('The Course Is UP-To-Market PERFECTLY !')
        
    # Question 5
    if RatesList[4] < 3 :
        BadConc.append("Your exams must be from course content, please")
    elif RatesList[4] >= 3 and RatesList[4] < 5:
        GoodConc.append('The Exam Content Is GOOD')
    else:
        GoodConc.append('The Exam Content Is PERFECT !')
        
    # Question 6
    if RatesList[5] < 3 :  
        BadConc.append("Please, be more organized")
    elif RatesList[5] >= 3 and RatesList[5] < 5:
        GoodConc.append('The instructor is organized')
    else:
        GoodConc.append('The instructor is REALLY organized!')
        
    # Question 7
    if RatesList[6] < 3 :
        BadConc.append("You need to improve the materials to increase students knowledge and skills in the subject matter")
    elif RatesList[6] >= 3 and RatesList[6] < 5:
        GoodConc.append('The materials are GOOD ')
    else:
        GoodConc.append('The materials are PERFECT !')
        
    # Question 8
    if RatesList[7] < 3 :
        BadConc.append("PLEASE treat your students better")
    elif RatesList[7] >= 3 and RatesList[7] < 5:
        GoodConc.append('The instructor treates the students GOOD ')
    else:
        GoodConc.append('The instructor treates the students  PERFECTLY!')
        
    # Question 9
    if RatesList[8] < 3 :
        BadConc.append("Work on yourself to be recommended in courses")
    elif RatesList[8] >= 3 and RatesList[8] < 5:
        GoodConc.append('the instructor is recommended KINDA Often!')
    else:
        GoodConc.append('the instructor is recommended SO MUCH !')
        
    # Join the Array Items     
    FGoodConc = "*"
    temp = list(map(str, GoodConc))
    GoodConc = FGoodConc.join(temp)

    FBadConc = "*"
    temp = list(map(str, BadConc))
    BadConc = FBadConc.join(temp)
    
#     return "sadasdasdasdasdasadsasdas"
    return GoodConc, BadConc

In [24]:
def SubjectRatePlot(df, subname, SubjectID):
    SubQuery = SubjectIndexing(df)
     
    RatesList = []
    avg = 0
    for column in SubQuery.columns[1:]:
            SubQuery['Total'] = SubQuery[column]*SubQuery['value']
            rate = round(SubQuery['Total'].sum()/SubQuery[column].sum())
            RatesList.append(rate)
            avg += rate
    SubjectFinalRate = avg / 9 
    
    # Update Datebase With SubjectFinalRate
    cursor.execute("""
    UPDATE Subjects
    SET Rate = ?
    WHERE SubjectId = ?
    """ ,(SubjectFinalRate, SubjectID))

    conn.commit()

    # Conclosion Function
    GoodConc, BadConc = SubjectConclsion(RatesList)

    conn.commit()
    
    # Plot Function
    SubjectIframe = SubjectPlot(df,subname)
    
    # DON'T REMOVE THE RETURN 
    return SubjectFinalRate, GoodConc, BadConc, SubjectIframe

## Subject Calcolation Start Point

## Prof Main Page Plot

In [25]:
def profRate(subjectAnalysisDf,profId):
    ratee = round(subjectAnalysisDf.query('ProfessorId == @profId').AnalysisRate.mean())
    
    # Update Datebase With SubjectFinalRate
    cursor.execute("""
    UPDATE Professors
    SET Rate = ?
    WHERE Id = ?
    """ ,(ratee, profId))
    
    conn.commit()
    
    return ratee
             

In [26]:
def ProfRatePlot(subjectAnalysisDf,profId):
    # Rate Calcolation
    rate = profRate(subjectAnalysisDf,profId)
    
    # Get the Prof's Data 
    Prof1 = subjectAnalysisDf.groupby(['ProfessorId','SubjectId','Name','AnalysisDate'],as_index = False)\
    ['AnalysisRate'].mean().query('ProfessorId == @profId').sort_values(by='AnalysisDate')

    # Plot Set-Up
    Prof = ex.line(Prof1,x='AnalysisDate',y='AnalysisRate',color='Name', markers=True,animation_frame='Name',text='Name')
    Prof.update_layout(
        yaxis_range =[0, 6],
        title=f'Prof performance over time',
        title_x=0.5,
        xaxis_title="Survey Date",
        yaxis_title="Over All Rate",
        legend_title="Subject ID",
        font=dict(
            family="Open Sans", 
            size=14,
            color="RebeccaPurple"
        ),
        showlegend=False
    )

    Prof.update_traces(
        textposition='top center',
        hovertemplate="<br>".join([
            "Faculty Name : %{y}",
            "Department Name: %{x}",

        ])
    )


    # Change Animation Time
    Prof.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 3000

    # Disable Controllers
    Prof.layout.xaxis.fixedrange = True
    Prof.layout.yaxis.fixedrange = True

    # Pring The Plot
#   Prof.show()

    # Generate Subject Plot IFRAME
    SubIframe = tls.get_embed(py.plot(Prof, auto_open=False, filename = profId))

    cursor.execute("""
    UPDATE Professors
    SET IFrame = ?
    WHERE Id = ?
    """ ,(SubIframe, profId))
    
    conn.commit()
    
    # return The Iframe
    return SubIframe, rate

# Department Plot and Rate

In [27]:
def DepartmentRatePlot(df, DepID):
    # Rate function
    rate = depRate(df,DepID)
    
    # Plot Code
    df = df.query('DepartmentId ==@DepID').sort_values(by=['Name','AnalysisDate'])
    Dep2 = ex.line(df, x = 'AnalysisDate', y = 'AnalysisRate',\
                   animation_frame = 'Name', markers=True, text='Name',\
                  color='Name')
    Dep2.update_layout(
        yaxis_range =[0, 6],
        title=f'Department performance over time',
        title_x=0.5,
        xaxis_title="Survey Date",
        yaxis_title="Over All Rate",
        legend_title="Subject ID",
        font=dict(
            family="Open Sans", 
            size=14,
            color="RebeccaPurple"
        ),
        showlegend=False
    )

    Dep2.update_traces(
        textposition='top center',
        hovertemplate="<br>".join([
            "Faculty Name : %{y}",
            "Department Name: %{x}",

        ])
    )


    # Change Animation Time
    Dep2.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 3000

    # Disable Controllers
    Dep2.layout.xaxis.fixedrange = True
    Dep2.layout.yaxis.fixedrange = True
    
    # Generate Subject Plot IFRAME
    SubIframe = tls.get_embed(py.plot(Dep2, auto_open=False, filename = DepID))
    
    cursor.execute("""
    UPDATE Departments
    SET IFrame = ?
    WHERE DepartmentId = ?
    """ ,(SubIframe, DepID))
    
    conn.commit()
    # Print The Iframe
    return(SubIframe,rate)

In [28]:
def depRate(subjectAnalysisDf,departmentId):
    rate = round(subjectAnalysisDf.query('DepartmentId == @departmentId').AnalysisRate.mean())
    
    # Update Datebase With SubjectFinalRate
    cursor.execute("""
    UPDATE Departments
    SET Rate = ?
    WHERE DepartmentId = ?
    """ ,(rate, departmentId))
    
    conn.commit()
    
    return rate
             

In [29]:
def facultyRatePlot(departmentAnalysisDf,faculityId):
    
    facrate = facultyRate(departmentAnalysisDf, faculityId)
    
    df = departmentAnalysisDf.query('FacultyId ==@faculityId').sort_values(by=['Name','AnalysisDate'])

    fac = ex.line(df, x = 'AnalysisDate', y = 'AnalysisRate',\
                   animation_frame = 'Name', markers=True, text='Name',\
                  color='Name')
    fac.update_layout(
        yaxis_range =[0, 6],
        title=f'Faculty performance over time',
        title_x=0.5,
        xaxis_title="Survey Date",
        yaxis_title="Over All Rate",
        legend_title="Subject ID",
        font=dict(
            family="Open Sans", 
            size=14,
            color="RebeccaPurple"
        ),
        showlegend=False
    )

    fac.update_traces(
        textposition='top center',
        hovertemplate="<br>".join([
            "Faculty Name : %{y}",
            "Department Name: %{x}",

        ])
    )


    # Change Animation Time
    fac.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 3000

    # Disable Controllers
    fac.layout.xaxis.fixedrange = True
    fac.layout.yaxis.fixedrange = True
        
    # Generate Subject Plot IFRAME
    SubIframe = tls.get_embed(py.plot(fac, auto_open=False, filename = faculityId))
    
    cursor.execute("""
    UPDATE Faculties
    SET IFrame = ?
    WHERE FacultyId = ?
    """ ,(SubIframe, faculityId))
    
    conn.commit()
#     Print The Iframe
    return(SubIframe,facrate)
    

In [30]:
def facultyRate(departmentAnalysisDf, faculityId):
    frate = round(departmentAnalysisDf.query('FacultyId == @faculityId').AnalysisRate.mean())
    
    # Update Datebase With SubjectFinalRate
    cursor.execute("""
    UPDATE Faculties
    SET Rate = ?
    WHERE FacultyId = ?
    """ ,(frate, faculityId))
    
    conn.commit()
    
    return frate

In [31]:
def universityRatePlot(FacultyAnalysesDf,universityId):
    
    uniirate = universityRate(FacultyAnalysesDf, universityId)
    
    df = FacultyAnalysesDf.query('UniversityId ==@universityId').sort_values(by=['Name','AnalysisDate'])
    df['Name'] = df['Name'].apply(lambda x: x.split("Of")[1])

    uni = ex.line(df, x = 'AnalysisDate', y = 'AnalysisRate',\
                   animation_frame = 'Name', markers=True, text='Name',\
                  color='Name')
    uni.update_layout(
        yaxis_range =[0, 6],
        title=f'University performance over time',
        title_x=0.5,
        xaxis_title="Survey Date",
        yaxis_title="Over All Rate",
        legend_title="University ID",
        font=dict(
            family="Open Sans", 
            size=14,
            color="RebeccaPurple"
        ),
        showlegend=False
    )

    uni.update_traces(
        textposition='top center',
        hovertemplate="<br>".join([
            "Faculty Name : %{y}",
            "Department Name: %{x}",
            "Survey Name: %{customdata[0]}"

        ])
    )


    # Change Animation Time
    uni.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 3000

    # Disable Controllers
    uni.layout.xaxis.fixedrange = True
    uni.layout.yaxis.fixedrange = True
        
    # Generate Subject Plot IFRAME
    SubIframe = tls.get_embed(py.plot(uni, auto_open=False, filename = universityId))
    
    cursor.execute("""
    UPDATE Universities
    SET IFrame = ?
    WHERE id = ?
    """ ,(SubIframe, universityId))
    
    conn.commit()
#   Print The Iframe
    return(SubIframe,uniirate)
    

In [32]:
def universityRate(FacultyAnalysesDf, universityId):
    urate = round(FacultyAnalysesDf.query('UniversityId == @universityId').AnalysisRate.mean())
    
    # Update Datebase With SubjectFinalRate
    cursor.execute("""
    UPDATE Universities
    SET Rate = ?
    WHERE id = ?
    """ ,(urate, universityId))
    
    conn.commit()
    
    return urate

# Project Start Point

In [34]:
def AllConclosions(GoodConc, BadConc,profId, subjId, QuestDate, questionnaireId):
    
    GoodConc = "No Good Conclosions" if len(GoodConc) <1 else GoodConc
    BadConc = "No Bad Conclosions" if len(BadConc) <1 else BadConc
    
    cursor.execute("""
    INSERT INTO SubjectConclusions
    (SubjectConclusionId
      ,Date
      ,GoodConclusion
      ,BadConclusion
      ,SubjectId
      ,QuestionnaireId)
      
      VALUES ( ?, ?, ?, ?, ?, ?)
    """ ,(uuid.uuid1(), QuestDate, GoodConc, BadConc, subjId, questionnaireId))
    
    conn.commit()
    
    cursor.execute("""
    INSERT INTO Recommendations
    (Date
      ,Content
      ,SubjectId
      ,ProfessorId
      ,RecommendationId)
      
      VALUES ( ?, ?, ?, ?, ?)
    """ ,(QuestDate, BadConc, subjId, profId, uuid.uuid1()))
    
    conn.commit()
    

# API Creation

In [36]:
# # TEST URL: http://127.0.0.1:8000/api/86F697D4-A762-44D6-8322-2C08C66F94E4/D0552B49-6E7D-4CED-8A30-62CE8066A2D4/84796C48-D538-4954-A98A-622DC5C9325A/15EE4163-B1D7-4FFD-9357-AE82B0CBA7A0/C1C46CC4-12C7-11EE-970C-105BADC84798/706870E9-E373-11ED-B719-105BADC84798

app = FastAPI()
@app.get("/api/{universityId}/{faculityId}/{departmentId}/{subjId}/{questionnaireId}/{profId}") #{profId}

def hello(universityId:str, faculityId:str, departmentId:str, subjId:str, questionnaireId:str, profId:str):
    # خلي بالملك من نوع المتغيرات الي جايه من ركوست مومن رقم ولا كلمه

    SubmitionsDf = pd.read_sql(
        """
        SELECT * FROM Submitions s
        JOIN Questionnaires q
        ON q.QuestionnaireId = s.QuestionnaireId
        JOIN Subjects sb 
        ON sb.SubjectId = q.SubjectId
        """, conn).query('QuestionnaireId == @questionnaireId')

    SubmitionsDf['EndDate'] = pd.to_datetime(SubmitionsDf['EndDate'])

    subname = SubmitionsDf.Name[0]
    QuestDate = str(SubmitionsDf.EndDate[0]).split()[0]

    ###############################################################################

    subjectAnalysisDf = pd.read_sql(
        """
        SELECT * FROM SubjectAnalyses sa
        JOIN Subjects s
        ON s.SubjectId = sa.SubjectId
        """, conn)

    SubjectId = subjectAnalysisDf.iloc[:,3]
    subjectAnalysisDf.drop(subjectAnalysisDf.columns[(3)], axis=1, inplace=True)
    subjectAnalysisDf['SubjectId'] = SubjectId

    subjectAnalysisDf['AnalysisDate'] = pd.to_datetime(subjectAnalysisDf['AnalysisDate'])

    ###############################################################################
    departmentAnalysisDf = pd.read_sql(
        """
        SELECT da.AnalysisRate, da.AnalysisDate, da.DepartmentId, f.FacultyId, d.Name, f.UniversityId FROM DepartmentAnalyses da
        JOIN Departments d
        ON d.DepartmentId = da.DepartmentId
        JOIN Faculties f
        ON f.FacultyId = d.FacultyId 
        """, conn)
    departmentAnalysisDf['AnalysisDate'] = pd.to_datetime(departmentAnalysisDf['AnalysisDate'])

    #####################################################################

    FacultyAnalysesDf = pd.read_sql(
        """
        SELECT fa.AnalysisRate, fa.AnalysisDate, f.UniversityId, f.Name FROM FacultyAnalyses fa
        JOIN Faculties f
        ON f.FacultyId = fa.FacultyId 
        JOIN Universities u 
        ON u.id = f.UniversityId
        """, conn)
    FacultyAnalysesDf['AnalysisDate'] = pd.to_datetime(FacultyAnalysesDf['AnalysisDate'])
    #####################################################################

    SubjectFinalRate, GoodConc, BadConc ,SubjectIframe = SubjectRatePlot(SubmitionsDf, subname, subjId)
    AllConclosions(GoodConc, BadConc,profId, subjId, QuestDate, questionnaireId)
    ProfRatePlot(subjectAnalysisDf,profId)
    DepartmentRatePlot(subjectAnalysisDf, departmentId)
    facultyRatePlot(departmentAnalysisDf,faculityId)
    universityRate = universityRatePlot(FacultyAnalysesDf,universityId)
    SubjectMainPagePlot (subjectAnalysisDf, subjId)
    
    return "DoneZiZo"

if __name__ == "__main__":
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [7220]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:63723 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:63723 - "GET /favicon.ico HTTP/1.1" 404 Not Found


C:\Users\moham\AppData\Local\Temp\ipykernel_7220\2346520440.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  SubmitionsDf = pd.read_sql(
C:\Users\moham\AppData\Local\Temp\ipykernel_7220\2346520440.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  subjectAnalysisDf = pd.read_sql(
C:\Users\moham\AppData\Local\Temp\ipykernel_7220\2346520440.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  departmentAnalysisDf = pd.read_sql(
C:\Users\moham\AppData\Local\Temp\ipykernel_7220\2346520440.py:51: UserWarning: pandas only su

INFO:     127.0.0.1:63726 - "GET /api/86F697D4-A762-44D6-8322-2C08C66F94E4/D0552B49-6E7D-4CED-8A30-62CE8066A2D4/84796C48-D538-4954-A98A-622DC5C9325A/15EE4163-B1D7-4FFD-9357-AE82B0CBA7A0/C1C46CC4-12C7-11EE-970C-105BADC84798/706870E9-E373-11ED-B719-105BADC84798 HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [7220]
